# Char-based text generation with LSTM

In [1]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

In [2]:
corpus_path = 'corpus_oss.txt'
with open(corpus_path) as f:
    text = f.read()

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)
 
    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char
 
sequence, char_to_idx, idx_to_char = text_to_seq(text)

[' ', 'æ', 'а', 'ы', 'д', 'н', 'т', 'р', 'с', 'у', 'х', 'г', 'и', 'о', 'з', 'м', 'л', 'й', 'к', 'б', 'ъ', 'ф', 'ц', 'е', 'в', 'п', 'ж', 'ч', 'ь', 'я', 'ш', 'э', 'ю', 'i', 'щ', 'a', 'x', 'y', 'd', 'n', 'u', 'r', 't', 's', 'm', 'o', 'j', 'v', 'z', 'ё', 'g', 'l', 'k', 'b', 'c', 'f', 'e', '_', 'h', 'p', 'q', '\n', 'w', 'і', 'ӕ', 'â', 'é', 'š', 'è', 'ê', 'á', '²', 'ѳ', 'ў', 'ó', 'ʼ', 'ô', 'ā', 'à', 'ʰ', 'ʁ', 'ن', 'ا', 'ō', 'ö', 'ò', 'ѵ', 'ü', 'ý', 'ɒ', 'ʲ', 'ɢ', 'ʔ', 'í', 'ب', 'د', 'ه', 'ی', 'ش']


In [3]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [4]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    cur_len = 0
    while cur_len < prediction_len:
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
        if predicted_char == ' ':
            cur_len += 1
    
    return predicted_text

In [5]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 10000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

/home/artem/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Loss: 2.8254897212982177
 фарстæ ахъон сæрды хъастæ астæ фæрд сæлт æрхар фардзар кæр бад балард рардзæр харды бæрд ардзын мар уыр уард ралут кал куыр кæр кæрд кæрта фæстæ ахæр къады стыр къард стард æрстаг бæраг кæртæ адæр дæрдзыстæ хъæхс арæл бартæ тæрд соны раргæн бардæр æрастæ фалдæр алал дахæн ад ардзаг æрс хахæт æрмæй кæл кæл цæн бахи сæрдзын бардæд кон астæ къар кæд фадзар хъалд ахæсты ахæртæ алæр къартæ бахæст халæр хъарт стард хъарды хæрд ардзинæн бахъу æлрардæй ад хартæ æхæрдистæ сдзыл бæрд фæрд ахæрд æхсæр фард хъон сæрд кард ардзон ардæл хъæдзар адыр уар фарди мартæ уарди фард хъуын бард уар хæрд нард хъæрд ахæртæ бард бардзæг мæрды æрдын хъандаст алæрд сæнд хъи кæртæ рарты æрдзар растæ фырти æрхæр фад фардзын кахар фаста бахты æррæлд æрбахæн хæргæ ард ардæд цъард хæрд арды хон хард сæрстæ раррартæ астæй хард ууд ардзын цар хъард уырты стар иста бардзар анæрды хъæрд аргæл фæрд æрбард хæхтъ уар рар хартæ бахар ахæрд хъард кæрæн къæрд хъард хъахын фахæн арæнд фæрди рæрхæн хауы

Loss: 2.139642572402954
 дзар æвзар хъус къæрæг къуыд ализ фæндаг хъæлæсты хъазы саразын хъæуæг къусы бахæдзæр ныхъуыды хъæут ныхицæн хъусын хъæлæсты фæрсы хъуыды цæуын æрбацыдысты фæстон фæрт бараз фæлдзæг суагъта æрбацæуы бахæсты разын дзырд мæн æрбацыд баззад стысты мæсты сыгъд хъуыды фæзыдысты кæнæг къуыр къæрмæ ахаст куыста æрбахæстæ хъуыд æрбацыд хъуыст кæрæджы æрбад сыхъуыст бацæуын хъæлæсты хъуыды сæхи фæхъуыды кæндзæн уыдзы сæрын кæнджытæ кæрæджы сын къухтæ сæры бацæуын æрбацæуын хъæлæстæ æрдæхæн хъæлæсты уæлгъуытæ къахст кæндзæн зæхсы къуыд кодта æрхæстæг хъæут сæрдæг сараз фæстон сарæг таргъ фæлдæр кæрæг къухын хъуыды хъуыды хъæлæстæ ихъусы скъуыд æрцыдысты хъуыдз хъæлæсты мæнгæ кæстæр скъуыд баззаг кæнæг бардыг бахъуыст бацыдысты фæстад хъуыды сырд фыды мидæг сагътой аразын сагъта дзуры къард æрбацæуын хъазын хъæууыд сахъуыд араст æрбалыд къуырдта араз бахъуыд æнæмæй хъæлæс æрбацæуын хæстæг лæууыд саразын къары саразы къуыд кæрæг хъæлæс лæууыд хъæлæсты алыгъуыд куыстæй æрба

Loss: 2.011133522987366
 тæх сагъæс сæрын кæрæдзи цæуыл хъæдæй сых сæрыд æрхъæрдæм фæзынд уæлæ хъæлæс хъæдæй æрбахъæрон æрбахæццæ кæрæдз цыди сæхицæн фæндаг къуырын хъазы фæзынд хъæдæй æнæуын кæрæдзи æрхæццæ комы фæкаст фæзынд æрхъæлдзæн сырх хъуыды арæзт æртаст барæхст æрбауадз цæуынц кæрæдз кæрæдзи сырх куыст сæрæгъ æрцыдысты кæрæдзи кæндзынæн фæлдæхт æнæхъуыды фæлмæн хъæлæс хъуырдæг æрбахæццæ æрхъæл уырдыгæй кæрæдзи сымах хъæлæс кæрæдзи уæрæг сæрæг кæрæдзи фæнд кæрæдзи æрцæуын хъазы алæф архайд нырмæ бахъуыды бахъуыды хæрз хъæлæс зæрдæмæ кæрæдз къуырын уындзы кæрæдзи æппæт мæсты къах кæрæдзи фæрсæг кæнын кæмæ алынг сæрын хъæды хъазын æрцыдысты бахъуыды æртысты кæндзынæн хъуыды хъæлæс хъæлæс кæрæдзæн ком хъæдæй хъæлæс сагъæс ардæм ком сæрын къуырдзысты цæуын бахæццæ кæрæдзи хъуыды хъæумæ сырх бахъуыды æрбахæццæ ахъуыды æрбахъуыды сымах фæллад тах кæрæдз сахуыр æрхæссын кæнæг кæрæдз фæлæууыд кæрæдзи бахъуыды кæрæдзи бахъæлæс хъæлæс фæллад кæстæр кæрæдз дзырд æрхæссын баззад къæр хъуыд

Loss: 1.9303145074844361
 кæрадз къухтæ бакæнын радзырдта фæцæргæ зæххыл фæкæнын сарæзт саны фæуыд разынд исты уазæг алыгъд хъæлæс баззад дардта кæндзынæн æнхъæлмæ æркæндзынæн агуырд бахæссын уырдæм арæх хъæддаг дæуæн æрæдзынг ардæм арфæ райхъуыст æрбахастон радзырдта хæрзæг бафтыдта кæнут кæна хæрд уыдта æрбацыд аразын æрбадт уырны хъазы кæрæдзи фæуыдзæн фыды бафтыдта фæсте кæндзынæн бакаст арды ахæццæ бацыдысты фæлгæс хъуыст фæндыр ахуыргæнæг æрбахæссын æрбахаста æвдис скъæм фæдзæхст æрбадт бафæллад уырыссаг цырæгътæ фыдæлты ныхæс æрбады райсом хъуыды зæххы хъуыддæгтæ аразын кæрæдзи сæрыл арды хъуыст æрбадт фæцыд хъуыддаджы хъазын кæнут дзæгъæл фæндыр фæндагыл арæх бакаст сæрын хъæлдзæг сарæзт сахат ахсын бафæллад бахæццæ уыдта райдайы ахаст мæстæг дзаума комы хъазы фæлæууыд ныхасы уыдта арды ардæм къорд ауад хъæлæс фæлæууыд къухтæ хъал хъæлæс арæх саразын ахсын бафæрсын дзырд фæуадзын хъуырдыдта куыддæр арды æрбахаста хъæлæс фæлæууыд бахъуыды фæсивæд кæндзынæн бахъарын бахæццæ хæст 

...

Loss: 1.7109840488433838
 рахæстæг ахаст æрбацыд хъæды æрбахæццæ сæрæй фæлæуу хъæлæсæй мæнæй фæсте куысты бакæс фæзынд хъуыдытæ фæзынд фæлмæн хъазы бафæрс æнæуи ныххаста æрбадти æфсæддон дзырдтой сом æрбацыд хъæрæй бамбарын сывæллон арæх æрæвæрдта кæсынц базыдта райсом хъуыдытæ фæлæууыд сывæллæтты кæрæдзи бахъахъхъæнын хъæлæс ныххæццæ райсом ныхъхъус æрбацæуын фæндагыл раздæр сæрдар æрбацыд хъуыддаджы фæлтæр уадз хъуыдытæ сарæзт хæст арæзт къæхтæ фæлæууыд фæсарæн къухæй æргъæвдзысты байдыдта хъæбул сылгоймаг райста хъуыдытæ фæзынд фæндагыл базыдта туг хæрд фæлæууыд райсом сæрибар сылгоймаг æрбадтысты æрбахаста æрбахуыдта фæлæуу барæй фæллой æрцæуын хæст æрбадт арæх ныфс æрбахъуыды хъæуы дзырдтой æрбадт фæлæууыд фæндагыл цæуыл уарзон хъæлæсæй æрбахаста æрбацыд æрбацыд хъæбысы хъазы арæх фæлæууыд æрбадти кæрæдзийы æрбацыд архайдта хæрз бахæццæ æрбахæццæ æрбакæнын æрцæуын дзырдтон æрбахæццæ бахудт арæзт сыгъдæг арф хъæлæс кæрæдзийы бахау сырх сыгъдæг уарзын арты бафтыд куынæ бакæн араст ц

Loss: 1.727476544380188
 раздæр кæрон хæрз фæллад хуымæтæджы цæу æрбадти бахъуыд базонын æрбадт æнæхъæн бахæццæ алчи фæндагыл арæх фæдæ дыс хъуыддаджы æрбацыд уыдзысты бакæн адæймаджы уацмыстæ къухты хъуыддæгтæ хъуыдытæ хъазы къах къуыри мард хъуыддаджы куысты фæзынд хъазы бафæрс æхсызгон сæрдар уæлæ ныххудт науæд фæлæуу æрбацыд тæрсын саразын хъæумæ æрхауд иуæй сæры къæхты хъæбул æрбацыд бахъуыды сарæхст цæуыл æрбадт хъæлæс хъæугæрон сырд фæзынд æрбахæццæ сылгоймаг æрбахæссын базонын ахсын хъазы бæрзæй бакæн æрбахаста бадт цæрын арæхст арæзт æрбадт фыды хъæрæй хъазы æрбадт мæрдтæм бадын æрцыди æрбацыд æхсæн арвы сарæзт базыдта аразын уæлæ фæхæцыд саразын хъуыдытæ разынд æрбацæуы сæры куырдта æрбахæццæ уæлæмæ фæсивæд райсом æрбацæуын райсом арæх сылгоймæгтæ сылгоймаг фæлтæр сырд ахуыргæнæг хъæбатыр цæстæнгас ныххæцыд арæх æрбацæуын сæрыл æрхауд алæууыд тыхст фæллад кæрæдзийы арæхст искæй ныллæг фæллад къухтæ арæх сабыр лæууыд сæрыл æнæхъæн къуыри бакæн фæлмæн байдыдта хъусын райсом фæх

Loss: 1.7288131618499756
 æрцыди сылгоймаг æрбадтысты кæрæдзийæн хъæды уадзын сарæзт æрбацыд салдат аразын ардыгæй хъæуы æрбадтысты ахуыргæнджытæ бахъуыд бакæн фæзынд фæлæуу æрхауд æрцыди хъæды алыхæст хъуыддаджы хæсты æрцæуын бахъуыд фæдзæхст раздæр бандон сырд арф кæрæдзийы фæндагыл æрбацыд бахъуыды фæсивæд къухтæ фæсивæд аразын уазæг хъæлæс фæсивæд хъæлæсæй бафтыдта бахæццæ уæлæ кæндзысты фæстæг сырх дзæнгæрæг дзыхæй кæндзынæн фæллой куыдз кæрæдзи бахъуыд сыгъдæг бахаудта сырх æрбацыд араст хæстæ кæндзынæн æнæмæнг сывæллон æрбацæуын хæсты фæхъазын саразын æрбадт сылгоймаг бахаудтой хъуыст сæрыл фæстагмæ бахъуыды уыдонæн хатыр хъæуы ахуыргæнæг сауджын уыдзынæ сæрæй фæдзæхст бацæуын æрбацæуы ахаст сарæзт хъæлæс бахъардта зæрдæмæ махæн ацæуын æрбакæнын хæсты хъуыдытæ фæнд хъуыддæгтæ баст сырд райста хъуыддаджы бадын рахиз арæх сæры дзинад куырдта уæлæ бадын куыддæр хъæубæстæ бафтыдта комкоммæ бахъуыды хæрз арæх арæхст бацыди кæсæг æрбадти хъару дзырдтой сæрыл сырд сырх кæрæдзимæ сырд з

Loss: 1.7346151971817017
 хъæлæс æрæнцад къæсæр фæрсын райста уæлæмæ цæрын тæрсын уарзон фæрсын разæй ныхасы фæкæнын фæкаст хъæлæс бахъæуын хъуыддаг æрцыди æргом дзых уæлæмæ æрбацыд фæрсын æрдзы сæры алыварс фæрсын ардыгæй хъазы хъус кæндзынæн сыгъдæг фæллад фæлтæрд аразын æрхæццæ хæрдмæ æрбадт дзырдтæ хъæлæс хуыссæны сæры хъæуынц æрæджиау æрбадти раздæр кæрæдзийæн фæлмæн æрбахæццæ хæрз хъарм кæндзынæн сæрыл сылгоймаг хъазын байдыдта бауадз хъуыдытæ сæрыл æнæхъæн фæуыдзынæ сахат æрбацæуын æрхаудта фæлæуу уæлæмæ æрбадти комы сагъæс цæрын æрдзы разынд фæрс сæры æрбахæццæ базыдта фæндагыл кæндзынæн хъуыддаджы уыдонæн кæстæр хъæбул хуыссы арт хъуыдытæ хъæлæсæй æрбацыд къухтæ сæрыл фæкæнын мидæг æрбахæццæ æрбацыд бакæн сæрыл сæрдар мæхæдæг уарзон галиу хъæлдзæг арæх сæртæ тæрсын арæхст фыдæлты хъæд сырд хъарм бахъуыд хъуыддæгтæ сæрыл базыр кæндзæн хæстон æрцыди бамбæрста сыхæгтæ фыды хæдзармæ æрбахаста фæрсын фæлæуу фæллой мард кæрæдзийы фæрсын фæкæс æрцыди къуым баззадысты аразын уадз кæрæ

Loss: 1.7146626138687133
 риу сæрæй цæуылнæ хъавыд æрбацыд къухтæ æрбахæццæ базыдта хъæумæ бахæццæ сæрдар хъæбул раздæр райгуырд арæх ныхæс райсом архайдтой райсом æрбадти фæдæн карз куырдта æрбацыд арæх хъазы ахаста фæндыр сабыр кæрæдзи æнæбары хъуыдытæ æрбацыд уарзондзинад хъæбул дæлæ уазал бахъуыды арт бахæццæ сæхимæ базыдта тарст баурæдта æрцыди бафæрс æрбадт сагъæс гæрæх фæзынд кæрæдзи фæзынд базыдтон бадт сывæллæттæ сослан хъæлдзæг сыгъдæг хъуыдытæ бахæццæ кæрæдзийы арæх æрдæг араст хæрд хъæбул арæхст хъуыдытæ хъæбæр хæст дæлæ хъæды бад къухæй бахъуыды æрхæсс аразын сæрдар хæст хæрз æрдæг бахæццæ фæрстæ хуыссæн бахудт уарзондзинад цæуыл фæстæг сæрыл фæцæуы хъæбысы къуыбыр æрбакодтой кæндзынæн æнæхæдæг хъæбул æрæвæрдта хъæдæй æрбацыд къæрцц арæзт æрхаста æрбадт хъайтар æрбацæуы æрбахæццæ арф хъазы бацæуын æрхаста фæндагыл саразын бахудт арæх сырх цæстæй бахъуыды дзырдта бамбæрста дзабыр фæллойгæнæг æрбацыд разынд бакæс фæндагыл арф хъуыддаджы æрбацыд æрбадт фæлæуу бахæццæ аразын с

In [7]:
torch.save(model.state_dict(), 'generator.model')

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.load_state_dict(torch.load('generator.model'))
model.to(device)

TextRNN(
  (encoder): Embedding(99, 128)
  (lstm): LSTM(128, 128, num_layers=2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=128, out_features=99, bias=True)
)

In [ ]:
with open(corpus_path) as f:
    texts = f.readlines()
    
generated_texts = []
for text in tqdm(texts):
    splitted_text = text.split()
    generated_text = ''
    for i, word in enumerate(splitted_text):
        if i % 200 == 0:
            generated_text += evaluate(model, char_to_idx, idx_to_char, temp=0.3, prediction_len=200, start_text=word)
        
    generated_texts.append(generated_text)
    
generated_path = 'generated_corpus_oss.txt'
with open(generated_path, 'w') as f:
    for generated_text in generated_texts:
        f.write(generated_text + '\n') 

 52%|█████████████████▎               | 629/1201 [15:20:40<16:57:26, 106.72s/it]